Kreditentscheidungen
====================

Fallstudie ins Deutsche übersetzt, ursprünglich hier abgerufen: [https://fairlearn.org/v0.12/auto_examples/plot_credit_loan_decisions.html#sphx-glr-auto-examples-plot-credit-loan-decisions-py](https://fairlearn.org/v0.12/auto_examples/plot_credit_loan_decisions.html#sphx-glr-auto-examples-plot-credit-loan-decisions-py)


Package Imports
===============


In [ ]:
import warnings

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from fairlearn.metrics import (
    MetricFrame,
    count,
    equalized_odds_difference,
    false_negative_rate,
    false_positive_rate,
    selection_rate,
)
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.reductions import EqualizedOdds, ExponentiatedGradient

warnings.simplefilter("ignore")

rand_seed = 1234
np.random.seed(rand_seed)

Fairness-Überlegungen bei Kreditentscheidungen
============================================

Fairness und Kreditvergabe in den USA
-------------------------------------

2019 erhielt Apple in den sozialen Medien Kritik, nachdem das neu eingeführte *Apple Card*-Produkt scheinbar Männern höhere Kreditlimits im Vergleich zu Frauen angeboten hat `nedlund2019apple`{.interpreted-text role="footcite"}. In mehreren Fällen stellten verheiratete Paare fest, dass der Ehemann ein Kreditlimit erhielt, das 10-20-mal höher war als das der Ehefrau, selbst wenn das Paar gemeinsame Vermögenswerte hatte.

Aus regulatorischer Sicht unterliegen Finanzinstitute, die innerhalb der Vereinigten Staaten tätig sind, *gesetzlichen Vorschriften*, die Diskriminierung aufgrund von [Rasse, Geschlecht oder anderen geschützten Klassen :footcite:`uscode2011title15chapter41subchapteriv]{.title-ref} verbieten. Mit der zunehmenden Verbreitung automatisierter Entscheidungssysteme im Bereich der Finanzkreditvergabe haben Experten Bedenken geäußert, ob diese Systeme bestehende Ungleichheiten in der Finanzkreditvergabe verschärfen könnten.

Obwohl die beiden Konzepte miteinander verflochten sind, ist algorithmische Fairness nicht dasselbe wie Antidiskriminierungsgesetz. Ein KI-System kann mit Antidiskriminierungsgesetzen konform sein und dennoch Fairness-bezogene Bedenken aufweisen. Andererseits können einige Fairness-Interventionen nach Antidiskriminierungsgesetzen illegal sein. :footcite:`Xiang2019legalcompatibility`{.interpreted-text role="cts"} diskutieren die Kompatibilitäten und Trennungen zwischen Antidiskriminierungsgesetzen und algorithmischen Fairness-Konzepten. In dieser Fallstudie konzentrieren wir uns auf Fairness im Finanzdienstleistungsbereich anstelle der Einhaltung finanzieller Antidiskriminierungsvorschriften.

Ernst & Young (EY) Fallstudie
============================

In dieser Fallstudie zielen wir darauf ab, die Arbeit in einem Whitepaper `dudik2020assessing`{.interpreted-text role="footcite"}, mitverfasst von *Microsoft* und *EY*, zur Minderung geschlechtsbezogener Leistungsunterschiede bei Finanzkreditentscheidungen zu replizieren. In ihrer Analyse zeigten Microsoft und EY, wie Fairlearn verwendet werden kann, um Unfairness im Kreditentscheidungsprozess zu messen und zu mindern.

Mit einem Datensatz von Kreditausgangsergebnissen (ob eine Person mit einem Kredit ausgefallen ist) trainieren wir ein Fairness-unbewusstes Modell, um die Wahrscheinlichkeit vorherzusagen, dass eine Person bei einem bestimmten Kredit ausfällt. Wir verwenden das Fairlearn-Toolkit, um die Fairness unseres Modells anhand mehrerer Metriken zu bewerten. Schließlich führen wir zwei Unfairness-Minderungsstrategien an unserem Modell durch und vergleichen die Ergebnisse mit unserem ursprünglichen Modell.

Da der im Whitepaper verwendete Datensatz nicht öffentlich verfügbar ist, werden wir ein semi-synthetisches Merkmal in einen bestehenden öffentlich verfügbaren Datensatz einführen, um die im ursprünglichen Datensatz gefundenen Ergebnisunterschiede zu replizieren.

Kreditentscheidungsdatensatz
---------------------------

Wie bereits erwähnt, werden wir den ursprünglichen Kreditausgangsdaten nicht verwenden können und stattdessen mit einem öffentlich verfügbaren Datensatz von Kreditkartenausfällen in Taiwan aus dem Jahr 2005 arbeiten. Dieser Datensatz stellt binäre Kreditkartenausgangsergebnisse für 30.000 Antragsteller mit Informationen zu der Zahlungshistorie und den Rechnungsstellungen eines Antragstellers über einen Zeitraum von sechs Monaten von April 2005 bis September 2005 sowie demografischen Informationen wie *Geschlecht*, *Alter*, *Familienstand* und *Bildungsniveau* des Antragstellers dar. Eine vollständige Zusammenfassung der Merkmale finden Sie unten:

| Merkmale                                                                   | Beschreibung                                       |
| --------------------------------------------------------------------------| --------------------------------------------------|
| sex, education, marriage, age                                              | demografische Merkmale                             |
| pay\_0, pay\_2, pay\_3, pay\_4, pay\_5, pay\_6                             | Rückzahlungsstatus (ordinal)                       |
| bill\_amt1, bill\_amt2, bill\_amt3, bill\_amt4, bill\_amt5, bill\_amt\_6   | Rechnungsbetrag (Taiwan-Dollar)                    |
| pay\_amt1, pay\_amt2, pay\_amt3, pay\_amt4, pay\_amt5, pay\_amt6           | vorheriger Rechnungsbetrag (Taiwan-Dollar)         |
| default payment next month                                                 | Ausfalldaten (1 = JA, 0 = NEIN)                    |

Stellen Sie sich vor, wir sind ein Datenwissenschaftler bei einem Finanzinstitut, der damit beauftragt ist, ein Klassifikationsmodell zu entwickeln, das vorhersagt, ob ein Antragsteller bei einem Privatkredit ausfallen wird. Eine positive Vorhersage des Modells bedeutet, dass der Antragsteller bei dem Kredit ausfallen würde. *Bei einem Kredit ausfallen* bedeutet, dass der Kunde innerhalb eines 30-Tage-Fensters keine Zahlungen leistet und der Kreditgeber rechtliche Schritte gegen den Kunden einleiten kann.

Obwohl wir keinen Datensatz mit Kredit-Ausfallhistorie haben, verfügen wir über diesen Datensatz mit Kreditkartenzahlungshistorie. Wir nehmen an, dass Kunden, die monatliche Kreditkartenzahlungen pünktlich leisten, als kreditwürdiger gelten und daher weniger wahrscheinlich bei einem Privatkredit ausfallen.

**Entscheidungspunkt: Aufgabenstellung**

-   **Zahlungsausfall bei einer Kreditkartenzahlung** kann als Proxy dafür angesehen werden, dass ein Antragsteller möglicherweise kein guter andidat für einen Privatkredit ist.
-   Da die meisten Kunden bei ihren Kreditkartenzahlungen nicht ausfallen, müssen wir dieses Klassenungleichgewicht während unseres Modellierungsprozesses berücksichtigen.

Da die Daten im Speicher gelesen werden, ändern wir die Spalte `PAY_0` in `PAY_1`, um die Benennung konsistenter mit den anderen Spalten zu gestalten. Darüber hinaus wird die Zielvariable `default payment next month` zu `default` geändert, um die Verbosität zu reduzieren.


In [ ]:
data_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
dataset = (
    pd.read_excel(io=data_url, header=1)
    .drop(columns=["ID"])
    .rename(columns={"PAY_0": "PAY_1", "default payment next month": "default"})
)

dataset.shape

In [ ]:
dataset.head()

Aus der [Datensatzbeschreibung
:footcite:`yeh2009comparisons]{.title-ref}, sehen wir, dass es drei
kategorische Merkmale gibt:

-   `SEX`: Geschlecht des Antragstellers (als binäres Merkmal)
-   `EDUCATION`: Höchstes Bildungsniveau, das vom Antragsteller erreicht wurde.
-   `MARRIAGE`: Familienstand des Antragstellers.

In [ ]:
categorical_features = ["SEX", "EDUCATION", "MARRIAGE"]

for col_name in categorical_features:
    dataset[col_name] = dataset[col_name].astype("category")

Y, A = dataset.loc[:, "default"], dataset.loc[:, "SEX"]
X = pd.get_dummies(dataset.drop(columns=["default", "SEX"]))

A_str = A.map({1: "male", 2: "female"})

Datensatz-Ungleichgewichte
==========================

Bevor wir mit dem Training eines Klassifikationsmodells beginnen, möchten wir den Datensatz auf Eigenschaften untersuchen, die später im Modellierungsprozess zu fairness-bezogenen Schäden führen könnten. Insbesondere werden wir uns auf die Verteilung des sensiblen Merkmals `SEX` und des Ziellabels `default` konzentrieren.

Im Rahmen einer explorativen Datenanalyse lassen Sie uns die Verteilung unseres sensiblen Merkmals `SEX` untersuchen. Wir sehen, dass 60% der Kreditnehmer als [weiblich]{.title-ref} und 40% als [männlich]{.title-ref} gekennzeichnet wurden, sodass wir uns keine Sorgen über ein Ungleichgewicht in diesem Merkmal machen müssen.

In [ ]:
A_str.value_counts(normalize=True)

Next, let\'s explore the distribution of the *loan default rate* `Y`. We
see that around 78% of individuals in the dataset do not default on
their credit loan. While the target label does not display extreme
imbalance, we will need to account for this imbalance in our modeling
section. As opposed to the *sensitive feature* `SEX`, an imbalance in
the target label may result in a classifier that over-optimizes for the
majority class. For example, a classifier that predicts an applicant
will not default would achieve an accuracy of 78%, so we will use the
`balanced_accuracy` score as our evaluation metric to counteract the
label imbalance.


In [ ]:
Y.value_counts(normalize=True)

Fügen Sie synthetisches Rauschen hinzu, das mit dem Ergebnis und dem Geschlecht zusammenhängt
==============================================================================================

Für diese Fallstudie fügen wir ein synthetisches Merkmal `Interest` hinzu, das eine Korrelation zwischen dem `SEX`-Label eines Antragstellers und dem `default`-Ergebnis einführt. Der Zweck dieses Merkmals besteht darin, Ergebnisunterschiede, die im ursprünglichen Datensatz vorhanden sind, zu replizieren. Wir können dieses `Interest`-Merkmal als den *Zinssatz* für den Antragsteller betrachten. Wenn der Antragsteller eine Geschichte von Ausfällen bei Kreditkartenzahlungen hat, wird die Bank dem Antragsteller einen höheren Zinssatz anbieten. Wir nehmen auch an, dass, weil Banken historisch gesehen hauptsächlich an Männer verliehen haben, es für diese Antragsteller weniger Unsicherheit (oder Varianz) im *Zinssatz* gibt.

Um die oben genannten Überlegungen widerzuspiegeln, wird das `Interest`-Merkmal aus einer *Gaußschen Verteilung* mit den folgenden Kriterien gezogen:

-   Wenn *Männlich*, ziehe `Interest` aus
    $\mathcal{N}(2 \cdot \text{Default}, 1)$
-   Wenn *Weiblich*, ziehe `Interest` aus
    $\mathcal{N}(2 \cdot \text{Default}, 2)$

Dieses Merkmal wird aus einer *Gaußschen Verteilung* zur rechnerischen Einfachheit gezogen.


In [ ]:
X.loc[:, "Interest"] = np.random.normal(loc=2 * Y, scale=A)

Überprüfen Sie, ob dies zu Unterschieden im naiven Modell führt
===================================================
    
Nachdem wir unser synthetisches Merkmal erstellt haben, lassen Sie uns überprüfen, wie dieses neue Merkmal mit unserem *sensitive_feature* `Sex` und unserem Ziellabel `default` interagiert. Wir sehen, dass für beide Geschlechter das Merkmal `Interest` höher ist bei Personen, die ihren Kredit ausgefallen haben.


In [ ]:
fig, (ax_1, ax_2) = plt.subplots(ncols=2, figsize=(10, 4), sharex=True, sharey=True)
X["Interest"][(A == 1) & (Y == 0)].plot(
    kind="kde", label="Payment on Time", ax=ax_1, title="INTEREST for Men"
)
X["Interest"][(A == 1) & (Y == 1)].plot(kind="kde", label="Payment Default", ax=ax_1)
X["Interest"][(A == 2) & (Y == 0)].plot(
    kind="kde",
    label="Payment on Time",
    ax=ax_2,
    legend=True,
    title="INTEREST for Women",
)
X["Interest"][(A == 2) & (Y == 1)].plot(
    kind="kde", label="Payment Default", ax=ax_2, legend=True
).legend(bbox_to_anchor=(1.6, 1))

Training eines ersten Modells
=============================

In diesem Abschnitt werden wir ein Fairness-unbewusstes Modell auf den Trainingsdaten trainieren. Aufgrund der Ungleichgewichte im Datensatz werden wir jedoch zunächst die Trainingsdaten neu sampeln, um einen neuen ausgeglichenen Trainingsdatensatz zu erstellen.



In [ ]:
def resample_training_data(X_train, Y_train, A_train):
    """Down-sample the majority class in the training dataset to produce a
    balanced dataset with a 50/50 split in the predictive labels.

    Parameters:
    X_train: The training split of the features
    Y_train: The training split of the target labels
    A_train: The training split of the sensitive features

    Returns:
    Tuple of X_train, Y_train, A_train where each dataset has been re-balanced.
    """
    negative_ids = Y_train[Y_train == 0].index
    positive_ids = Y_train[Y_train == 1].index
    balanced_ids = positive_ids.union(np.random.choice(a=negative_ids, size=len(positive_ids)))

    X_train = X_train.loc[balanced_ids, :]
    Y_train = Y_train.loc[balanced_ids]
    A_train = A_train.loc[balanced_ids]
    return X_train, Y_train, A_train

In [ ]:
X_train, X_test, y_train, y_test, A_train, A_test = train_test_split(
    X, Y, A_str, test_size=0.35, stratify=Y
)

X_train, y_train, A_train = resample_training_data(X_train, y_train, A_train)

An diesem Punkt werden wir einen *radient-boosted tree classifier* mithilfe des `lightgbm`-Pakets auf dem ausgeglichenen Trainingsdatensatz trainieren. Bei der Bewertung des Modells werden wir den unausgeglichenen Testdatensatz verwenden.

In [ ]:
lgb_params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.03,
    "num_leaves": 10,
    "max_depth": 3,
    "random_state": rand_seed,
    "n_jobs": 1,
    "verbose": -1,
}

estimator = Pipeline(
    steps=[
        ("preprocessing", StandardScaler()),
        ("classifier", lgb.LGBMClassifier(**lgb_params)),
    ]
)

estimator.fit(X_train, y_train)

Wir berechnen die *binary prediction* und die *prediciton probabilities* für die Testdaten.

In [ ]:
Y_pred_proba = estimator.predict_proba(X_test)[:, 1]
Y_pred = estimator.predict(X_test)

Am *ROC Score* können wir sehen, dass das Modell anscheinend zwischen *true positives* und *false positives* zu unterscheiden vermag. Das war auch so zu erwarten, denn `INTEREST` ist ein stark diskrimminierendes Feature für die Klassifikation.


In [ ]:
roc_auc_score(y_test, Y_pred_proba)

Merkmalswichtigkeit des unveränderten Klassifikators
====================================================
    
Als Modellvalidierungsprüfung lassen Sie uns die Merkmalswichtigkeiten unseres Klassifikators untersuchen. Wie erwartet hat unser synthetisches Merkmal `INTEREST` die höchste Merkmalswichtigkeit, da es konstruktionsbedingt stark mit der Zielvariablen korreliert.



In [ ]:
lgb.plot_importance(
    estimator.named_steps["classifier"],
    height=0.6,
    title="Feature Importance",
    importance_type="gain",
    max_num_features=15,
)

Fairness-Bewertung des unveränderten Modells
===========================================

Nachdem wir unser erstes Fairness-unbewusstes Modell trainiert haben, führen wir nun unsere Fairness-Bewertung für dieses Modell durch. Bei der Durchführung einer Fairness-Bewertung gibt es drei Hauptschritte, die wir durchführen möchten:

1.  Identifizieren, wer geschädigt wird.
2.  Identifizieren der Arten von Schäden, die wir erwarten.
3.  Definieren von Fairness-Metriken basierend auf den erwarteten Schäden.

Wer wird geschädigt?
---------------------

Basierend auf dem Vorfall mit der *Apple* Kreditkarte, der zu Beginn dieses Notebooks erwähnt wurde, glauben wir, dass das Modell fälschlicherweise vorhersagen könnte, dass Frauen bei der Kreditvergabe ausfallen werden. Das System könnte Frauen unfairerweise weniger Kredite zuweisen und Männern übermäßig Kredite gewähren.


Arten von erlebten Schäden
==========================

Beim Diskutieren von Fairness in KI-Systemen ist der erste Schritt zu verstehen, welche Arten von Schäden wir erwarten, dass das System erzeugen könnte. Mit der `Schadenstaxonomie im Fairlearn Benutzerhandbuch <types_of_harms>`{.interpreted-text role="ref"}, erwarten wir, dass dieses System *Zuweisungsschäden* erzeugt. Darüber hinaus erwarten wir auch die langfristigen Auswirkungen auf die Kreditwürdigkeit einer Person, wenn eine Person nicht in der Lage ist, einen erhaltenen Kredit zurückzuzahlen oder wenn sie für einen Kreditantrag abgelehnt wird. Ein *Zuweisungsschaden* tritt auf, wenn ein KI-System Ressourcen, Möglichkeiten oder Informationen erweitert oder zurückhält. In diesem Szenario erweitert oder hält das KI-System finanzielle Ressourcen von Individuen zurück oder verteilt sie aus. Ein Rückblick auf historische Vorfälle zeigt, dass diese Art von automatisierten Kreditentscheidungsystemen möglicherweise unfair basierend auf dem Geschlecht diskriminieren.

**Negative Auswirkungen auf die Kreditwürdigkeit**

Eine sekundäre Schadensart, die in Kreditentscheidungen etwas einzigartig ist, sind die langfristigen Auswirkungen auf die Kreditwürdigkeit einer Person. In den Vereinigten Staaten ist eine [FICO-Kreditwürdigkeitsscore](https://www.investopedia.com/terms/c/credit_score.asp) eine Zahl zwischen 300 und 850, die die *Kreditwürdigkeit* eines Kunden darstellt. Die Kreditwürdigkeit eines Antragstellers wird von vielen Finanzinstituten für Kreditentscheidungen verwendet. Die Kreditwürdigkeit eines Antragstellers steigt in der Regel nach einer erfolgreichen Rückzahlung eines Kredits und sinkt, wenn der Antragsteller den Kredit nicht zurückzahlt.

Bei der Beantragung eines Kredits gibt es drei Hauptausgänge:

1.  Die Person erhält den Kredit und zahlt den Kredit zurück. In diesem Szenario erwarten wir, dass die Kreditwürdigkeit der Person als Ergebnis der erfolgreichen Rückzahlung des Kredits steigt.
2.  Die Person erhält den Kredit, fällt aber beim Kredit aus. In diesem Szenario wird die Kreditwürdigkeit der Person aufgrund der Nichtzahlung des Kredits drastisch sinken. Im Modellierungsprozess ist dieses Ergebnis mit einem **falschen Negativ** verbunden (das Modell sagt voraus, dass die Person den Kredit zurückzahlen wird, die Person ist jedoch nicht erfolgreich).
3.  In bestimmten Ländern, wie den Vereinigten Staaten, erhält eine Person nach einer *harten Anfrage* zu ihrer Kredithistorie einen kleinen Rückgang (bis zu fünf Punkte) ihrer Kreditwürdigkeit. Wenn der Antragsteller einen Kredit beantragt, aber keinen erhält, wird der kleine Rückgang seiner Kreditwürdigkeit seine Fähigkeit beeinträchtigen, sich erfolgreich für einen zukünftigen Kredit zu bewerben. Im Modellierungsprozess ist dieses Ergebnis mit der **Auswahlrate** verbunden (der Anteil der positiven Vorhersagen, die vom Modell ausgegeben werden).

**Verhinderung von Vermögensakkumulation**

Eine weitere Schadensart, die wir in diesem Szenario erwarten, sind die langfristigen Auswirkungen der *Ablehnung von Krediten an Antragsteller, die den Kredit erfolgreich zurückgezahlt hätten*. Durch die Erhaltung eines Kredits kann ein Antragsteller ein Haus kaufen, ein Unternehmen gründen oder eine andere wirtschaftliche Aktivität ausüben, die er sonst nicht tun könnte. Diese Ergebnisse sind mit **falschen Positivfehlern** verbunden, bei denen das Modell vorhersagt, dass ein Antragsteller beim Kredit ausfallen wird, die Person den Kredit jedoch erfolgreich zurückgezahlt hätte. In den Vereinigten Staaten hat die Praxis des Redlining `peyton2020redlining`{.interpreted-text role="footcite"}, bei der Hypothekarkredite und andere Finanzdienstleistungen überwiegend schwarzen oder anderen Minderheitengemeinschaften verweigert werden, zu einer erheblichen rassischen Vermögenslücke zwischen weißen und schwarzen Amerikanern geführt. Obwohl die Praxis des Redlining 1968 mit dem *Fair Housing Act* verboten wurde, spiegeln die langfristigen Auswirkungen dieser Praktiken `jan2018redlining`{.interpreted-text role="footcite"} den Mangel an wirtschaftlicher Investition in schwarze Gemeinschaften wider, und schwarze Antragsteller werden im Vergleich zu weißen Amerikanern häufiger Kredite verweigert.

Fairness-Metriken basierend auf Schäden definieren
---------------------------------------------------

Nachdem wir die relevanten Schäden identifiziert haben, die wir erwarten, dass die Nutzer erleben, können wir unsere Fairness-Metriken definieren. Zusätzlich zu den Metriken werden wir die Unsicherheit um jede Metrik mithilfe von *benutzerdefinierten Funktionen* berechnen, um den *Standardfehler* für jede Metrik auf dem $\alpha=0.95$ Konfidenzniveau zu berechnen.


In [ ]:
def compute_error_metric(metric_value, sample_size):
    """Compute standard error of a given metric based on the assumption of
    normal distribution.

    Parameters:
    metric_value: Value of the metric
    sample_size: Number of data points associated with the metric

    Returns:
    The standard error of the metric
    """
    metric_value = metric_value / sample_size
    return 1.96 * np.sqrt(metric_value * (1.0 - metric_value)) / np.sqrt(sample_size)


def false_positive_error(y_true, y_pred):
    """Compute the standard error for the false positive rate estimate."""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return compute_error_metric(fp, tn + fp)


def false_negative_error(y_true, y_pred):
    """Compute the standard error for the false negative rate estimate."""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return compute_error_metric(fn, fn + tp)


def balanced_accuracy_error(y_true, y_pred):
    """Compute the standard error for the balanced accuracy estimate."""
    fpr_error, fnr_error = false_positive_error(y_true, y_pred), false_negative_error(
        y_true, y_pred
    )
    return np.sqrt(fnr_error**2 + fpr_error**2) / 2


fairness_metrics = {
    "count": count,
    "balanced_accuracy": balanced_accuracy_score,
    "balanced_acc_error": balanced_accuracy_error,
    "selection_rate": selection_rate,
    "false_positive_rate": false_positive_rate,
    "false_positive_error": false_positive_error,
    "false_negative_rate": false_negative_rate,
    "false_negative_error": false_negative_error,
}

Wähle eine Selektion an Metriken aus, um Information-Overload zu vermeiden.

In [ ]:
metrics_to_report = [
    "balanced_accuracy",
    "false_positive_rate",
    "false_negative_rate",
]

Um die aufgeschlüsselten Leistungsmetriken zu berechnen, werden wir das `MetricFrame`-Objekt innerhalb der Fairlearn-Bibliothek verwenden. Wir werden unser Dictionary von Metriken `fairness_metrics` sowie unsere Testlabels `y_test` und Testvorhersagen `Y_pred` übergeben. Zusätzlich übergeben wir die *sensitive_features* `A_test`, um unsere Modellergebnisse aufzuschlüsseln.

Instanziieren des MetricFrame für das unveränderte Modell


In [ ]:
metricframe_unmitigated = MetricFrame(
    metrics=fairness_metrics,
    y_true=y_test,
    y_pred=Y_pred,
    sensitive_features=A_test,
)

metricframe_unmitigated.by_group[metrics_to_report]

metricframe_unmitigated.difference()[metrics_to_report]

metricframe_unmitigated.overall[metrics_to_report]

In [ ]:
def plot_group_metrics_with_error_bars(metricframe, metric, error_name):
    """Plot the disaggregated metric for each group with an associated
    error bar. Both metric and the error bar are provided as columns in the
    provided MetricFrame.

    Parameters
    ----------
    metricframe : MetricFrame
        The MetricFrame containing the metrics and their associated
        uncertainty quantification.
    metric : str
        The metric to plot
    error_name : str
        The associated standard error for each metric in metric

    Returns
    -------
    Matplotlib Plot of point estimates with error bars
    """
    grouped_metrics = metricframe.by_group
    point_estimates = grouped_metrics[metric]
    error_bars = grouped_metrics[error_name]
    lower_bounds = point_estimates - error_bars
    upper_bounds = point_estimates + error_bars

    x_axis_names = [str(name) for name in error_bars.index.to_flat_index().tolist()]
    plt.vlines(
        x_axis_names,
        lower_bounds,
        upper_bounds,
        linestyles="dashed",
        alpha=0.45,
    )
    plt.scatter(x_axis_names, point_estimates, s=25)
    plt.xticks(rotation=0)
    y_start, y_end = np.round(min(lower_bounds), decimals=2), np.round(
        max(upper_bounds), decimals=2
    )
    plt.yticks(np.arange(y_start, y_end, 0.05))
    plt.ylabel(metric)

In [ ]:
plot_group_metrics_with_error_bars(
    metricframe_unmitigated, "false_positive_rate", "false_positive_error"
)

In [ ]:
plot_group_metrics_with_error_bars(
    metricframe_unmitigated, "false_negative_rate", "false_negative_error"
)

In [ ]:
metricframe_unmitigated.by_group[metrics_to_report].plot.bar(
    subplots=True, layout=[1, 3], figsize=[12, 4], legend=None, rot=0
)

Schließlich berechnen wir die `equalized_odds_difference` für dieses unveränderte Modell. Die `equalized_odds_difference` ist das Maximum der `false_positive_rate_difference` und der `false_negative_rate_difference`. In unserem Kreditvergabe-Kontext führen sowohl *false_negative_rate_disparities* als auch *false_positive_rate_disparities* zu fairness-bezogenen Schäden. Daher versuchen wir, beide Metriken zu minimieren, indem wir die `equalized_odds_difference` minimieren.

In [ ]:
balanced_accuracy_unmitigated = balanced_accuracy_score(y_test, Y_pred)
equalized_odds_unmitigated = equalized_odds_difference(y_test, Y_pred, sensitive_features=A_test)

Eine wichtige Annahme hier ist, dass wir davon ausgehen, dass *falsche Positive* und *falsche Negative* für jede Gruppe gleichermaßen nachteilige Kosten haben. In der Praxis würden wir einen Gewichtungsmechanismus entwickeln, um jedem *falschen Negativ* und *falschen Positiv* Ereignis ein Gewicht zuzuweisen.

Minderung von Unfairness in ML-Modellen
======================================

Im vorherigen Abschnitt haben wir Unterschiede in der Leistung des Modells in Bezug auf `SEX` identifiziert. Insbesondere haben wir festgestellt, dass das Modell eine deutlich höhere `false_negative_rate` und `false_positive_rate` für die als `female` gekennzeichneten Antragsteller im Vergleich zu den als `male` gekennzeichneten produziert. Im Kontext des Kreditentscheidungs-Szenarios bedeutet dies, dass das Modell Kredite für *Frauen*, die den Kredit hätten zurückzahlen können, unterzuweisen scheint, aber Kredite für *Frauen*, die ihren Kredit ausfallen lassen, zu überzuweisen scheint.

In diesem Abschnitt werden wir Strategien zur Minderung der Leistungsunterschiede, die wir in unserem unveränderten Modell gefunden haben, diskutieren. Wir werden zwei verschiedene Minderungsstrategien anwenden:

-   *Postprocessing*: Beim Postprocessing-Ansatz werden die Ausgaben eines trainierten Klassifikators transformiert, um ein bestimmtes Fairness-Kriterium zu erfüllen.

-   *Reductions*: Beim Reductions-Ansatz nehmen wir eine Modellklasse und erstellen iterativ eine Folge von Modellen, die eine bestimmte Fairness-Beschränkung optimieren. Im Vergleich zum *Postprocessing*-Ansatz wird die Fairness-Beschränkung während des Modelltrainings erfüllt, anstatt danach.

Postprocessing-Minderungen: ThresholdOptimizer
-----------------------------------------------

Im Fairlearn-Paket wird die *Postprocessing*-Minderung durch den `ThresholdOptimizer`-Algorithmus angeboten, gemäß :footcite`hardt2016equality`{.interpreted-text role="cts"}. Der `ThresholdOptimizer` nimmt ein bestehendes (möglicherweise vortrainiertes) Machine Learning-Modell, dessen Vorhersagen als Bewertungsfunktion dienen, um separate Schwellenwerte für jede *sensitive_features* Gruppe zu identifizieren. Der `ThresholdOptimizer` optimiert eine spezifizierte Zielmetrik (in unserem Fall `balanced_accuracy`) unter Einhaltung einer bestimmten Fairness-Beschränkung ([equalized_odds]{.title-ref}), was zu einer geschwellenwerteten Version des zugrunde liegenden Machine Learning-Modells führt.

Um unseren `ThresholdOptimizer` zu instanziieren, müssen wir unsere Fairness-Beschränkung als Modellparameter spezifizieren. Da sowohl die Unterschiede in der `false_negative_rate` als auch in der `false_positive_rate` in unserem Szenario zu realen Schäden führen, werden wir versuchen, die `equalized_odds`-Differenz als unsere *Fairness-Beschränkung* zu minimieren.


In [ ]:
postprocess_est = ThresholdOptimizer(
    estimator=estimator,
    constraints="equalized_odds",  # Optimize FPR and FNR simultaneously
    objective="balanced_accuracy_score",
    prefit=True,
    predict_method="predict_proba",
)

Eine wesentliche Einschränkung des `ThresholdOptimizer` ist die Notwendigkeit sensibler Merkmale während des Trainings- und Vorhersagezeitpunkts. Wenn wir während der Vorhersagezeit keinen Zugriff auf die `sensitive_features` haben, können wir den `ThresholdOptimizer` nicht verwenden.

Wir übergeben `A_train` an die `fit`-Funktion mit dem Parameter `sensitive_features`.

In [ ]:
postprocess_est.fit(X=X_train, y=y_train, sensitive_features=A_train)

postprocess_pred = postprocess_est.predict(X_test, sensitive_features=A_test)

postprocess_pred_proba = postprocess_est._pmf_predict(X_test, sensitive_features=A_test)

Fairness Beurteilung des Post-processing Models
===========================================


In [ ]:
def compare_metricframe_results(mframe_1, mframe_2, metrics, names):
    """Concatenate the results of two MetricFrames along a subset of metrics.

    Parameters
    ----------
    mframe_1: First MetricFrame for comparison
    mframe_2: Second MetricFrame for comparison
    metrics: The subset of metrics for comparison
    names: The names of the selected metrics

    Returns
    -------
    MetricFrame : MetricFrame
        The concatenation of the two MetricFrames, restricted to the metrics
        specified.

    """
    return pd.concat(
        [mframe_1.by_group[metrics], mframe_2.by_group[metrics]],
        keys=names,
        axis=1,
    )

In [ ]:
bal_acc_postprocess = balanced_accuracy_score(y_test, postprocess_pred)
eq_odds_postprocess = equalized_odds_difference(
    y_test, postprocess_pred, sensitive_features=A_test
)

metricframe_postprocess = MetricFrame(
    metrics=fairness_metrics,
    y_true=y_test,
    y_pred=postprocess_pred,
    sensitive_features=A_test,
)

metricframe_postprocess.overall[metrics_to_report]

metricframe_postprocess.difference()[metrics_to_report]

Vergleichen wir nun die Leistung unseres *thresholded* Klassifikators mit dem ursprünglichen *nicht-mitigierten* Modell.


In [ ]:
compare_metricframe_results(
    metricframe_unmitigated,
    metricframe_postprocess,
    metrics=metrics_to_report,
    names=["Unmitigated", "PostProcess"],
)

metricframe_postprocess.by_group[metrics_to_report].plot.bar(
    subplots=True, layout=[1, 3], figsize=[12, 4], legend=None, rot=0
)

Wir sehen, dass der `ThresholdOptimizer`-Algorithmus eine viel geringere
Ungleichheit zwischen den beiden Gruppen im Vergleich zum *unveränderten* Modell erreicht.
Dies geht jedoch mit dem Nachteil einher, dass der `ThresholdOptimizer`
eine niedrigere `balanced_accuracy`-Score für *männliche* Antragsteller erreicht.

Reductions-Ansatz zur Minderung von Unfairness
===============================================
    
Im vorherigen Abschnitt nahmen wir ein Fairness-unbewusstes Modell und verwendeten den
`ThresholdOptimizer`, um die Entscheidungsgrenze des Modells zu transformieren, um
unsere Fairness-Beschränkungen zu erfüllen. Eine wesentliche Einschränkung des
`ThresholdOptimizer` ist die Notwendigkeit, während der Vorhersagezeit auf unser *sensibles Merkmal* zuzugreifen.
    
In diesem Abschnitt werden wir den *Reductions*-Ansatz von Agarwal et al.
(2018) `agarwal2018reductions`{.interpreted-text role="footcite"} verwenden, um
Modelle zu erstellen, die die Fairness-Beschränkung erfüllen, ohne während der Bereitstellungszeit Zugriff
auf die sensiblen Merkmale zu benötigen.
    
Der Hauptreduktion-Algorithmus in Fairlearn ist `ExponentiatedGradient`.
Der Algorithmus erstellt eine Folge von neu gewichteten Datensätzen und trainiert
den umschlossenen Klassifikator auf jedem dieser Datensätze neu. Dieser Neu-Trainingsprozess
garantiert die Auffindung eines Modells, das die Fairness-Beschränkungen erfüllt
und gleichzeitig die Leistungsmetrik optimiert.
    
Das von `ExponentiatedGradient` zurückgegebene Modell besteht aus mehreren inneren
Modellen, die von einem umschlossenen Estimator zurückgegeben werden.
    
Um ein `ExponentiatedGradient`-Modell zu instanziieren, übergeben wir zwei
Parameter:
    
-   einen Basis-`estimator` (Objekt, das das Training unterstützt)
-   Fairness `constraints` (Objekt vom Typ
    `fairlearn.reductions.Moment`{.interpreted-text role="class"})
    
Beim Übergeben einer Fairness-*Constraint* als `Moment` können wir einen
`epsilon`-Wert angeben, der die maximal erlaubte Differenz oder das Verhältnis
zwischen unserem größten und kleinsten Wert darstellt. Zum Beispiel bedeutet im folgenden Code,
`EqualizedOdds(difference_bound=epsilon)`, dass wir `EqualizedOdds` als unsere Fairness-Beschränkung verwenden und eine maximale
Differenz von `epsilon` zwischen unserem größten und kleinsten *equalized
odds*-Wert zulassen.

In [ ]:
def get_expgrad_models_per_epsilon(estimator, epsilon, X_train, y_train, A_train):
    """Instantiate and train an ExponentiatedGradient model on the
    balanced training dataset.

    Parameters
    ----------
    Estimator: Base estimator to contains a fit and predict function.
    Epsilon: Float representing maximum difference bound for the fairness Moment constraint

    Returns
    -------
    Predictors
        List of inner model predictors learned by the ExponentiatedGradient
        model during the training process.

    """
    exp_grad_est = ExponentiatedGradient(
        estimator=estimator,
        sample_weight_name="classifier__sample_weight",
        constraints=EqualizedOdds(difference_bound=epsilon),
    )
    # Is this an issue - Re-runs
    exp_grad_est.fit(X_train, y_train, sensitive_features=A_train)
    predictors = exp_grad_est.predictors_
    return predictors

Da der *Performance-Fairness Trade-off*, der vom `ExponentiatedGradient`-Modell gelernt wird, empfindlich auf unseren gewählten `epsilon`-Wert reagiert, können wir `epsilon` als einen *Hyperparameter* behandeln und über eine Reihe von potenziellen Werten iterieren. Hier werden wir zwei `ExponentiatedGradient`-Modelle trainieren, eines mit `epsilon=0.01` und das zweite mit `epsilon=0.02`, und die inneren Modelle, die durch jeden der Trainingsprozesse gelernt wurden, speichern.

In der Praxis empfehlen wir, kleinere Werte für `epsilon` in der Größenordnung der *Quadratwurzel* der Anzahl der Stichproben im Trainingsdatensatz zu wählen:
$\dfrac{1}{\sqrt{\text{numberSamples}}} \approx \dfrac{1}{\sqrt{25000}} \approx 0.01$


In [ ]:
epsilons = [0.01, 0.02]

In [ ]:
all_models = {}
for eps in epsilons:
    all_models[eps] = get_expgrad_models_per_epsilon(
        estimator=estimator,
        epsilon=eps,
        X_train=X_train,
        y_train=y_train,
        A_train=A_train,
    )

In [ ]:
for epsilon, models in all_models.items():
    print(f"For epsilon {epsilon}, ExponentiatedGradient learned {len(models)} inner models")

Hier können wir alle inneren Modelle sehen, die für jeden `epsilon`-Wert gelernt wurden. Beim `ExponentiatedGradient`-Modell spezifizieren wir einen `epsilon`-Parameter, der die maximale Ungleichheit in unserer Fairness-Metrik darstellt, die unser finales Modell erfüllen sollte. Zum Beispiel bedeutet ein `epsilon=0.02`, dass der Trainingswert der *equalized odds difference* des zurückgegebenen Modells höchstens `0.02` ist (wenn der Algorithmus konvergiert).

Überprüfung der inneren Modelle von ExponentiatedGradient
==========================================================

In vielen Situationen, aufgrund von Regulierung oder anderen technischen Einschränkungen, kann die zufällige Natur des `ExponentiatedGradient`-Algorithmus unerwünscht sein. Darüber hinaus führen die mehreren inneren Modelle des Algorithmus zu Herausforderungen bei der Modellinterpretierbarkeit. Eine mögliche Lösung, um diese Probleme zu vermeiden, besteht darin, eines der inneren Modelle auszuwählen und es stattdessen bereitzustellen.

Im vorherigen Abschnitt haben wir mehrere `ExponentiatedGradient`-Modelle auf unterschiedlichen `epsilon`-Niveaus trainiert und alle inneren Modelle gesammelt, die durch diesen Prozess gelernt wurden. Beim Auswählen eines geeigneten inneren Modells berücksichtigen wir Trade-offs zwischen unseren beiden interessierenden Metriken: *balanced error rate* und *equalized odds difference*. Da unser Fokus auf diesen beiden Metriken liegt, werden wir die Modelle herausfiltern, die in beiden Metriken von einem anderen Modell übertroffen werden (wir bezeichnen diese als *\"dominierten\"* Modelle), und nur die verbleibenden *\"nicht dominierten\"* Modelle plotten.


In [ ]:
def is_pareto_efficient(points):
    """Filter a NumPy Matrix to remove rows that are strictly dominated by
    another row in the matrix. Strictly dominated means the all the row values
    are greater than the values of another row.

    Parameters
    ----------
    Points: NumPy array (NxM) of model metrics.
        Assumption that smaller values for metrics are preferred.

    Returns
    -------
    Boolean Array
        Nx1 boolean mask representing the non-dominated indices.
    """
    n, m = points.shape
    is_efficient = np.ones(n, dtype=bool)
    for i, c in enumerate(points):
        if is_efficient[i]:
            is_efficient[is_efficient] = np.any(points[is_efficient] < c, axis=1)
            is_efficient[i] = True
    return is_efficient

In [ ]:
def filter_dominated_rows(points):
    """Remove rows from a DataFrame that are monotonically dominated by
    another row in the DataFrame.

    Parameters
    ----------
    Points: DataFrame where each row represents the summarized performance
            (balanced accuracy, fairness metric) of an inner model.

    Returns
    -------
    pareto mask: Boolean mask representing indices of input DataFrame that are not monotonically dominated.
    masked_DataFrame: DataFrame with dominated rows filtered out.

    """
    pareto_mask = is_pareto_efficient(points.to_numpy())
    return pareto_mask, points.loc[pareto_mask, :]

In [ ]:
def aggregate_predictor_performances(predictors, metric, X_test, Y_test, A_test=None):
    """Compute the specified metric for all classifiers in predictors.
    If no sensitive features are present, the metric is computed without
    disaggregation.

    Parameters
    ----------
    predictors: A set of classifiers to generate predictions from.
    metric: The metric (callable) to compute for each classifier in predictor
    X_test: The data features of the testing data set
    Y_test: The target labels of the teting data set
    A_test: The sensitive feature of the testing data set.

    Returns
    -------
    List of performance scores for each classifier in predictors, for the
    given metric.
    """
    all_predictions = [predictor.predict(X_test) for predictor in predictors]
    if A_test is not None:
        return [metric(Y_test, Y_sweep, sensitive_features=A_test) for Y_sweep in all_predictions]
    else:
        return [metric(Y_test, Y_sweep) for Y_sweep in all_predictions]

In [ ]:
def model_performance_sweep(models_dict, X_test, y_test, A_test):
    """Compute the equalized_odds_difference and balanced_error_rate for a
    given list of inner models learned by the ExponentiatedGradient algorithm.
    Return a DataFrame containing the epsilon level of the model, the index
    of the model, the equalized_odds_difference score and the balanced_error
    for the model.

    Parameters
    ----------
    models_dict: Dictionary mapping model ids to a model.
    X_test: The data features of the testing data set
    y_test: The target labels of the testing data set
    A_test: The sensitive feature of the testing data set.

    Returns
    -------
    DataFrame where each row represents a model (epsilon, index) and its
    performance metrics
    """
    performances = []
    for eps, models in models_dict.items():
        eq_odds_difference = aggregate_predictor_performances(
            models, equalized_odds_difference, X_test, y_test, A_test
        )
        bal_acc_score = aggregate_predictor_performances(
            models, balanced_accuracy_score, X_test, y_test
        )
        for i, score in enumerate(eq_odds_difference):
            performances.append((eps, i, score, (1 - bal_acc_score[i])))
    performances_df = pd.DataFrame.from_records(
        performances,
        columns=["epsilon", "index", "equalized_odds", "balanced_error"],
    )
    return performances_df

In [ ]:
performance_df = model_performance_sweep(all_models, X_test, y_test, A_test)

In [ ]:
performance_subset = performance_df.loc[:, ["equalized_odds", "balanced_error"]]

In [ ]:
mask, pareto_subset = filter_dominated_rows(performance_subset)

performance_df_masked = performance_df.loc[mask, :]

Jetzt plotten wir die Performance-Tradeoffs zwischen all unseren Modellen.

In [ ]:
for index, row in performance_df_masked.iterrows():
    bal_error, eq_odds_diff = row["balanced_error"], row["equalized_odds"]
    epsilon_, index_ = row["epsilon"], row["index"]
    plt.scatter(bal_error, eq_odds_diff, color="green", label="ExponentiatedGradient")
    plt.text(
        bal_error + 0.001,
        eq_odds_diff + 0.0001,
        f"Eps: {epsilon_}, Idx: {int(index_)}",
        fontsize=10,
    )
plt.scatter(
    1.0 - balanced_accuracy_unmitigated,
    equalized_odds_unmitigated,
    label="UnmitigatedModel",
)
plt.scatter(1.0 - bal_acc_postprocess, eq_odds_postprocess, label="PostProcess")
plt.xlabel("Weighted Error Rate")
plt.ylabel("Equalized Odds")
plt.legend(bbox_to_anchor=(1.85, 1))

Wir sehen, dass der `ThresholdOptimizer`-Algorithmus eine viel geringere
Ungleichheit zwischen den beiden Gruppen im Vergleich zum *unveränderten* Modell erreicht.
Dies geht jedoch mit dem Nachteil einher, dass der `ThresholdOptimizer`
eine niedrigere `balanced_accuracy`-Score für *männliche* Antragsteller erreicht.

Reductions-Ansatz zur Minderung von Unfairness
===============================================

Im vorherigen Abschnitt nahmen wir ein Fairness-unbewusstes Modell und verwendeten den
`ThresholdOptimizer`, um die Entscheidungsgrenze des Modells zu transformieren, um
unsere Fairness-Beschränkungen zu erfüllen. Eine wesentliche Einschränkung des
`ThresholdOptimizer` ist die Notwendigkeit, während der Vorhersagezeit auf unser *sensibles Merkmal* zuzugreifen.

In diesem Abschnitt werden wir den *Reductions*-Ansatz von Agarwal et al.
(2018) `agarwal2018reductions`{.interpreted-text role="footcite"} verwenden, um
Modelle zu erstellen, die die Fairness-Beschränkung erfüllen, ohne während der Bereitstellungszeit Zugriff
auf die sensiblen Merkmale zu benötigen.

Der Hauptreduktion-Algorithmus in Fairlearn ist `ExponentiatedGradient`.
Der Algorithmus erstellt eine Folge von neu gewichteten Datensätzen und trainiert
den umschlossenen Klassifikator auf jedem dieser Datensätze neu. Dieser Neu-Trainingsprozess
garantiert die Auffindung eines Modells, das die Fairness-Beschränkungen erfüllt
und gleichzeitig die Leistungsmetrik optimiert.

Das von `ExponentiatedGradient` zurückgegebene Modell besteht aus mehreren inneren
Modellen, die von einem umschlossenen Estimator zurückgegeben werden.

Um ein `ExponentiatedGradient`-Modell zu instanziieren, übergeben wir zwei
Parameter:

-   einen Basis-`estimator` (Objekt, das das Training unterstützt)
-   Fairness `constraints` (Objekt vom Typ
    `fairlearn.reductions.Moment`{.interpreted-text role="class"})

Beim Übergeben einer Fairness-*Beschränkung* als `Moment` können wir einen
`epsilon`-Wert angeben, der die maximal erlaubte Differenz oder das Verhältnis
zwischen unserem größten und kleinsten Wert darstellt. Zum Beispiel bedeutet im folgenden Code,
`EqualizedOdds(difference_bound=epsilon)`, dass wir `EqualizedOdds` als unsere Fairness-Beschränkung verwenden und eine maximale
Differenz von `epsilon` zwischen unserem größten und kleinsten *equalized
odds*-Wert zulassen.

Da der *Performance-Fairness Trade-off*, der vom `ExponentiatedGradient`-Modell gelernt wird, empfindlich auf unseren gewählten `epsilon`-Wert reagiert, können wir `epsilon` als einen *Hyperparameter* behandeln und über eine Reihe von potenziellen Werten iterieren. Hier werden wir zwei `ExponentiatedGradient`-Modelle trainieren, eines mit `epsilon=0.01` und das zweite mit `epsilon=0.02`, und die inneren Modelle, die durch jeden der Trainingsprozesse gelernt wurden, speichern.

In der Praxis empfehlen wir, kleinere Werte für `epsilon` in der Größenordnung der *Quadratwurzel* der Anzahl der Stichproben im Trainingsdatensatz zu wählen:
$\dfrac{1}{\sqrt{\text{numberSamples}}} \approx \dfrac{1}{\sqrt{25000}} \approx 0.01$

Hier können wir alle inneren Modelle sehen, die für jeden `epsilon`-Wert gelernt wurden. Beim `ExponentiatedGradient`-Modell spezifizieren wir einen `epsilon`-Parameter, der die maximale Ungleichheit in unserer Fairness-Metrik darstellt, die unser finales Modell erfüllen sollte. Zum Beispiel bedeutet ein `epsilon=0.02`, dass der Trainingswert der *equalized odds difference* des zurückgegebenen Modells höchstens `0.02` ist (wenn der Algorithmus konvergiert).

Überprüfung der inneren Modelle von ExponentiatedGradient
==========================================================

In vielen Situationen, aufgrund von Regulierung oder anderen technischen Einschränkungen, kann die zufällige Natur des `ExponentiatedGradient`-Algorithmus unerwünscht sein. Darüber hinaus führen die mehreren inneren Modelle des Algorithmus zu Herausforderungen bei der Modellinterpretierbarkeit. Eine mögliche Lösung, um diese Probleme zu vermeiden, besteht darin, eines der inneren Modelle auszuwählen und es stattdessen bereitzustellen.

Im vorherigen Abschnitt haben wir mehrere `ExponentiatedGradient`-Modelle auf unterschiedlichen `epsilon`-Niveaus trainiert und alle inneren Modelle gesammelt, die durch diesen Prozess gelernt wurden. Beim Auswählen eines geeigneten inneren Modells berücksichtigen wir Trade-offs zwischen unseren beiden interessierenden Metriken: *balanced error rate* und *equalized odds difference*. Da unser Fokus auf diesen beiden Metriken liegt, werden wir die Modelle herausfiltern, die in beiden Metriken von einem anderen Modell übertroffen werden (wir bezeichnen diese als die *\"dominierten\"* Modelle), und nur die verbleibenden *\"nicht dominierten\"* Modelle plotten.


In [ ]:
def filter_models_by_unmitigiated_score(
    all_models,
    models_frames,
    unmitigated_score,
    performance_metric="balanced_error",
    fairness_metric="equalized_odds",
    threshold=0.01,
):
    """Filter out models whose performance score is above the desired
    threshold. Out of the remaining model, return the models with the best
    score on the fairness metric.

    Parameters
    ----------
    all_models: Dictionary (Epsilon, Index) mapping (epilson, index number) pairs to a Model object
    models_frames: A DataFrame representing each model's performance and fairness score.
    unmitigated_score: The performance score of the unmitigated model.
    performance_metric: The model performance metric to threshold on.
    fairness_metric: The fairness metric to optimize for
    threshold: The threshold padding added to the :code:`unmitigated_score`.

    """
    # Create threshold based on balanced_error of unmitigated model and filter
    models_filtered = models_frames.query(
        f"{performance_metric} <= {unmitigated_score + threshold}"
    )
    best_row = models_filtered.sort_values(by=[fairness_metric]).iloc[0]
    # Choose the model with smallest equalized_odds difference
    epsilon, index = best_row[["epsilon", "index"]]
    return {
        "model": all_models[epsilon][index],
        "epsilon": epsilon,
        "index": index,
    }

In [ ]:
best_model = filter_models_by_unmitigiated_score(
    all_models,
    models_frames=performance_df,
    unmitigated_score=(1.0 - balanced_accuracy_unmitigated),
    threshold=0.015,
)

print(
    f"Epsilon for best model: {best_model.get('epsilon')}, Index number: {best_model.get('index')}"
)
inprocess_model = best_model.get("model")

Jetzt haben wir unser bestes inneres Modell ausgewählt, lassen Sie uns die Vorhersagen des Modells auf dem Testdatensatz sammeln und die relevanten Leistungsmetriken berechnen.


In [ ]:
y_pred_inprocess = inprocess_model.predict(X_test)

bal_acc_inprocess = balanced_accuracy_score(y_test, y_pred_inprocess)
eq_odds_inprocess = equalized_odds_difference(y_test, y_pred_inprocess, sensitive_features=A_test)

In [ ]:
metricframe_inprocess = MetricFrame(
    metrics=fairness_metrics,
    y_true=y_test,
    y_pred=y_pred_inprocess,
    sensitive_features=A_test,
)

In [ ]:
metricframe_inprocess.difference()[metrics_to_report]

metricframe_inprocess.overall[metrics_to_report]

metricframe_inprocess.by_group[metrics_to_report].plot.bar(
    subplots=True, layout=[1, 3], figsize=[12, 4], legend=None, rot=0
)

Diskussion der Performance und des Trade-Offs
======================================

Jetzt haben wir zwei verschiedene Fairness-aware Modelle mithilfe des *Postprocessing*-Ansatzes und des *Reductions*-Ansatzes trainiert. Lassen Sie uns die Leistung dieser Modelle mit unserem ursprünglichen Fairness-unbewussten Modell vergleichen.


In [ ]:
metric_error_pairs = [
    ("balanced_accuracy", "balanced_acc_error"),
    ("false_positive_rate", "false_positive_error"),
    ("false_negative_rate", "false_negative_error"),
]


def create_metricframe_w_errors(mframe, metrics_to_report, metric_error_pair):
    mframe_by_group = mframe.by_group.copy()
    for metric_name, error_name in metric_error_pair:
        mframe_by_group[metric_name] = mframe_by_group[metric_name].apply(lambda x: f"{x:.3f}")
        mframe_by_group[error_name] = mframe_by_group[error_name].apply(lambda x: f"{x:.3f}")
        mframe_by_group[metric_name] = mframe_by_group[metric_name].str.cat(
            mframe_by_group[error_name], sep="±"
        )
    return mframe_by_group[metrics_to_report]

Bericht über Modell Performance und Fehlerbalken für Metriken
=========================================================

**Unverändertes Modell**



In [ ]:
create_metricframe_w_errors(metricframe_unmitigated, metrics_to_report, metric_error_pairs)

metricframe_unmitigated.overall[metrics_to_report]

**ExponentiatedGradient Modell**


In [ ]:
create_metricframe_w_errors(metricframe_inprocess, metrics_to_report, metric_error_pairs)

**ThresholdOptimizer**


In [ ]:
metricframe_inprocess.overall[metrics_to_report]

create_metricframe_w_errors(metricframe_postprocess, metrics_to_report, metric_error_pairs)

metricframe_postprocess.overall[metrics_to_report]

We see both of our fairness-aware models yield a slight decrease in the
*balanced\_accuracy* for *male applicants* compared to our
fairness-unaware model. In the *reductions* model, we see a decrease in
the *false positive rate* for *female applicants*. This is accompanied
by an increase in the *false negative rate* for *male applicants*.
However overall, the *equalized odds difference* for the *reductions*
models is lower than that of the original fairness-unaware model.

Conclusion and Discussion
Wir sehen, dass beide unserer Fairness-aware Modelle eine leichte Abnahme der *balanced_accuracy* für *männliche Antragsteller* im Vergleich zu unserem Fairness-unbewussten Modell aufweisen. Im *Reductions*-Modell beobachten wir eine Abnahme der *false positive rate* für *weibliche Antragsteller*. Dies geht einher mit einer Zunahme der *false negative rate* für *männliche Antragsteller*. Insgesamt ist jedoch die *equalized odds difference* für die *Reductions*-Modelle niedriger als die des ursprünglichen Fairness-unbewussten Modells.

Fazit und Diskussion
====================

In dieser Fallstudie haben wir den Prozess der Bewertung eines Kreditentscheidungsmodells auf geschlechtsbezogene Leistungsunterschiede durchlaufen. Unsere Analyse folgt eng der Arbeit, die in dem Microsoft/EY Whitepaper `dudik2020assessing`{.interpreted-text role="footcite"} durchgeführt wurde, wo sie das *Fairlearn*-Toolkit verwendeten, um ein Fairness-unbewusstes baumbasiertes Modell zu auditieren. Wir haben *Postprocessing*- und *Reductions*-Minderungs-Techniken angewendet, um die *equalized odds difference* in unserem Modell zu mindern.

Durch den *Reductions*-Prozess haben wir ein Modell erstellt, das die *equalized odds difference* des ursprünglichen Modells reduziert, ohne eine drastische Erhöhung der *balanced error score*. Wenn dies ein echtes Modell wäre, das von einem Finanzinstitut entwickelt wird, würde die *balanced error score* eine Annäherung an die Rentabilität des Modells darstellen. Indem wir eine relativ ähnliche *balanced error score* beibehalten haben, haben wir ein Modell produziert, das die Rentabilität für das Unternehmen erhält und gleichzeitig fairere und gerechtere Ergebnisse für Frauen in diesem Szenario liefert.


References
==========

::: {.footbibliography}
:::


Aufgaben
=========================================================

### 1. **Pre-Processing Ansätze ausprobieren**
- **Beispiele für Techniken:**
  - **Reweighing:** Anpassung der Gewichtungen der Datenpunkte, um Ungleichgewichte in den sensiblen Merkmalen auszugleichen.
  - **Disparate Impact Remover:** Transformation der Daten, um den Einfluss sensibler Merkmale auf die Zielvariable zu reduzieren.
- **Aufgabe:** Wende eine Pre-Processing Methode an und bewerte deren Einfluss auf die Fairness- und Performance-Metriken des Modells.
(https://github.com/Trusted-AI/AIF360/blob/main/examples/tutorial_credit_scoring.ipynb)


### 2. **Untersuchung und Visualisierung zusätzlicher Biases im Datensatz**
- **Aufgabe:** 
  - Erstelle Visualisierungen (z.B. Histogramme, Boxplots) zur Analyse der Verteilung verschiedener Merkmale in Bezug auf das sensitive Merkmal `SEX` und das Ziellabel `default`.
  - Identifiziere mögliche Korrelationen oder Muster, die auf weitere Fairness-Herausforderungen hinweisen könnten.


### 3. **Vergleich verschiedener Fairness-Metriken**
- **Beispiele für Fairness-Metriken:**
  - **Demographic Parity (Demografische Parität)**
  - **Equal Opportunity (Gleiche Chancen)**
  - **Predictive Parity (Vorhersageparität)**
- **Aufgabe:** 
  - Berechne verschiedene Fairness-Metriken für die trainierten Modelle.
  - Diskutiere, wie sich die Wahl der Metrik auf die Bewertung der Modellfairness auswirkt.